In [114]:
import pandas as pd
import os
import re
from google import genai
from google.genai import types
import json
import time

In [115]:
INPUT_PATH = "./data_2/kaufhaus_produktdaten_test_100.xlsx"
SHEET_NAME = "Products"


In [20]:
def test_if_excel(input_path: str, sheet_name) -> pd.DataFrame:
    
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"File oder path not found: {input_path}")
    
    xls = pd.ExcelFile(input_path)
    
    if sheet_name not in xls.sheet_names:
        raise ValueError(f"sheet_name is not in this excel_file: {sheet_name}")
    
    else:
        kh_all_product_df = pd.read_excel(input_path, sheet_name)
        
    return kh_all_product_df

kh_produkte_df = test_if_excel(INPUT_PATH, SHEET_NAME)
print(kh_produkte_df.head(3))


  Produkt_ID           SKU        Marke                        Produktname  \
0     P10001  SKU-BHSAHXTH    RheinTech       RheinTech Schraubenset 4 Pro   
1     P10002  SKU-QE7CKW5N     AlpenFit                   AlpenFit Jeans 6   
2     P10003  SKU-QE7CKW5N  MünsterWerk  MünsterWerk Kaffeemaschine 5 Mini   

  Kategorie_1        Kategorie_2  \
0    Baumarkt  Schrauben & Nägel   
1  Bekleidung             Herren   
2  Elektronik    Haushaltsgeräte   

                                        Beschreibung  Preis_EUR  Preis_USD  \
0  RheinTech Schraubenset 4 Pro aus der Kategorie...     127.60     139.08   
1  AlpenFit Jeans 6 aus der Kategorie Bekleidung/...        NaN      93.36   
2  MünsterWerk Kaffeemashine 5 Mini aus der Kateg...     572.39     623.91   

   EUR_USD_Kurs_Referenz  ...  Leistung_W  Spannung_V         MHD  \
0                   1.09  ...         NaN         NaN  2026-03-02   
1                   1.09  ...         NaN         NaN         NaN   
2                   1.

In [118]:
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    
    df = df.copy()
    df.columns = [str(column).strip() for column in df.columns]
    df.columns = [re.sub(r"\s+", "  ", column) for column in df.columns]      
    
    return df

normalize_product_list_df = normalize_columns(kh_produkte_df)
print(normalize_product_list_df.head())



  Produkt_ID           SKU        Marke                        Produktname  \
0     P10001  SKU-BHSAHXTH    RheinTech       RheinTech Schraubenset 4 Pro   
1     P10002  SKU-QE7CKW5N     AlpenFit                   AlpenFit Jeans 6   
2     P10003  SKU-QE7CKW5N  MünsterWerk  MünsterWerk Kaffeemaschine 5 Mini   
3     P10004  SKU-JIUJV6OH  KüstenKlang         KüstenKlang Notebook 2 Pro   
4     P10005  SKU-JIUJV6OH    HansaHome            HansaHome Pfanne 2 Mini   

  Kategorie_1        Kategorie_2  \
0    Baumarkt  Schrauben & Nägel   
1  Bekleidung             Herren   
2  Elektronik    Haushaltsgeräte   
3  Elektronik           Computer   
4    Haushalt              Küche   

                                        Beschreibung  Preis_EUR  Preis_USD  \
0  RheinTech Schraubenset 4 Pro aus der Kategorie...     127.60     139.08   
1  AlpenFit Jeans 6 aus der Kategorie Bekleidung/...        NaN      93.36   
2  MünsterWerk Kaffeemashine 5 Mini aus der Kateg...     572.39     623.91   
3 

In [ ]:
normalize_product_list_df.columns[:1].astype(str).intduplicated(keep=False).sum()

np.int64(0)

In [116]:
def add_issues(issues, row, product_id, issue_type, field, severity, message):
    issues.append({
        "row": row,
        "product_id": product_id,
        "issue_type": issue_type,
        "field": field,
        "severity": severity,
        "message": message
    })
    

In [ ]:
def run_prechecks(df: pd.DataFrame) -> pd.DataFrame:
    all_issues = []
    
    dup_mask = df[df.duplicated(subset=["Produkt_ID"], keep=False)]
    for idx in df[dup_mask].index:
        add_issues(
            all_issues,
            idx,
            df.at[idx, "Produkt_ID"],
            "Doppelte Produkt_ID",
            "Produkt_ID",
            "high"
            "Produkt_ID kommt mehrfach vor"
        )
    
    for idx, row in df.iterrows():
        p_id = row.get(1) #should be the id
        
        if pd.isna(p_id):
            add_issues(
                all_issues,
                idx, 
                None, 
                "missing product id",
                "Product_ID", 
                "high",
                "es gibt keine existierende Produkt_ID"             
            )
            
    for sku, group in df.groupby("SKU"): #should be the SKU number
        if len(group) > 1:
            rows = list(group.index)
            for idx in rows:
                add_issues(
                        all_issues,
                        idx,
                        group.at[idx, "Produkt_ID"],
                        "Duplikate bei der SKU",
                        "SKU",
                        "high",
                        f"duplicate SKU with rows: {rows}"
                )
        
    for idx, row in df.iterrows():
        p_id = row.get("Produkt_ID")
        
        p_eur = row.get("Preis_EUR")
        p_usd = row.get("Preis_USD")
        
        price_eur = pd.to_numeric(row.get("Preis_EUR"),errors="coerce")
        price_usd = pd.to_numeric(row.get("Preis_USD"),errors="coerce")
        
        
        if pd.isna(p_eur) or price_eur <= 0:
            add_issues(
                all_issues,
                idx,
                p_id,
                "Preis in Euro <= 0",
                "Preis_EUR",
                "high",
                f"kein hinterlegter Preis: {idx, price_eur}" 
            )
            
        if pd.isna(row.get("Beschreibung")):
            print("hier")
            add_issues(
                all_issues,
                idx,
                p_id,
                "Beschreibung fehlt",
                "Beschreibung",
                "medium",
                f"keine Beschreibung hinterlegt: {idx}"
            )
            
               
            
            
    return pd.DataFrame(all_issues)

issues_df = run_prechecks(normalize_product_list_df)
print(issues_df.tail(1))




hier
     row product_id          issue_type      field severity  \
134   83     P10084  Preis in Euro <= 0  Preis_EUR     high   

                                   message  
134  kein hinterlegter Preis: (83, -743.9)  


In [ ]:
#def write_to_excel(df: pd.DataFrame) -> None:
#    OUTPUT_PATH = "./data_2/clean_kaufhaus_produktdaten_test_100.xlsx"
#    sheet_name = "products"
#    
#    with pd.ExcelWriter(OUTPUT_PATH, engine="openpyxl") as writer:
#         df.to_excel(writer, index=False, sheet_name=sheet_name)
#         
#write_to_excel(clean_product_list_df